In [30]:
!unzip -qq '/content/drive/MyDrive/그게뭐약/alyac_crop/17159.제스부틴정_crop_음각_녹색.zip'

In [31]:
import cv2 as cv
import numpy as np

def GetYValueListInContour(contour, xPos):
    '''
    xPos에 해당하는 y좌표를 contour에서 찾아 반환한다.
    '''
    yList = []
    for each in contour:
        currentXPos = each[0][0]
        currentYPos = each[0][1]
        if currentXPos == xPos:
            yList.append(currentYPos)

    return yList

def GetMinMaxPosInContour(contour, maskedImg):
    global minX,minY,maxX,maxY
    '''
    contour의 x,y 좌표에서 최대 최소, 중심 좌표를 반환한다.
    :param contour:
    :param cx:
    :param cy:
    :return:
    '''

    # 중심점을 구한다.
    M = cv.moments(contour)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    '''
    cv.circle(maskedImg, (cx, cy), 10, (0, 255, 255), -1)
    cv.imshow("maskedImg", maskedImg)
    cv.waitKey(0)
    '''
    # 초기값으로 외곽선의 중심 좌표를 넣어 준다.
    minX,minY,maxX,maxY = cx, cy, cx, cy

    for each in contour:
        currentX = each[0][0]
        currentY = each[0][1]
        if currentX < minX:
            minX = currentX
        elif currentX > maxX:
            maxX = currentX

        if currentY < minY:
            minY = currentY
        elif currentY > maxY:
            maxY = currentY

    return minX,minY,maxX,maxY, cx, cy

def GetPillContour(img_color, highThreshold = 50, showFlag = False):
    img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
    img_gray = cv.medianBlur(img_gray, 5)
    if showFlag:
        cv.imshow("Blur", img_gray)
        cv.waitKey(0)

    img_colorTmp = img_color.copy()

    low = 0
    img_canny = cv.Canny(img_gray, low, highThreshold)
    if showFlag:
        cv.imshow("img_canny", img_canny)
        cv.waitKey(0)

    adaptive = cv.adaptiveThreshold(img_canny, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 15, 0)

    # contour에서 구멍 난 경우를 대비
    kernel = np.ones((10, 10), np.uint8)
    closing = cv.morphologyEx(adaptive, cv.MORPH_CLOSE, kernel)
    contourInput = closing
    contours, hierarchy = cv.findContours(contourInput, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    maxIdx = 0
    idx = 0
    maxArea = 0.0

    for cnt in contours:
        area = cv.contourArea(cnt)
        if maxArea < area:
            maxIdx = idx
            maxArea = area
        idx = idx + 1

    cv.drawContours(img_colorTmp, contours, idx - 1, (0, 0, 255), 1)
    if showFlag:
        cv.imshow("img_colorTmp", img_colorTmp)
        cv.waitKey(0)

        # 알약 외곽선을 추출한다.
    #contour = contours[idx - 1]
    lastIdx = idx - 1
    return contours, lastIdx

def hangulFilePathImageRead ( filePath ) :

    stream = open( filePath.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    numpyArray = np.asarray(bytes, dtype=np.uint8)

    return cv.imdecode(numpyArray , cv.IMREAD_UNCHANGED)

def GetForegroundImgInfo(FilePath, showFlag = False):
    '''
    전경이미지와 전경이미지를 얻을떄 사용한 정보들을 반환
    :param FilePath:
    :param showFlag:
    :return:
    '''
    #import time
    #start = time.time()
    #print(FilePath)
    img_color = cv.imread(FilePath)
    #cv.imshow('ss', img_color)
    #cv.waitKey(0)
    #img_color = hangulFilePathImageRead(FilePath)
    img_color = cv.resize(img_color, (300,300))
    #cv.imshow('ss', img_color)
    #cv.waitKey(0)
    # 알약 외곽선을 추출한다.
    highThreshold = 100
    contours, lastIdx = GetPillContour(img_color =img_color, highThreshold = highThreshold, showFlag = showFlag)
    '''
    contourLen = len(contours)
    if showFlag:
        print("contour len : ")
        print(contourLen)
    loopCount = 0
    while contourLen > 1 :
        highThreshold -= 150
        contours, lastIdx = GetPillContour(img_color =img_color, highThreshold = highThreshold, showFlag = showFlag)
        contourLen = len(contours)
        if showFlag:
            print("contour len : ")
            print(contourLen)
        loopCount += 1
        if loopCount > 3 :
            raise Exception('contour Length is not 1')
    '''

    if contours is None:
        print("contour is not found!!")
    # 검정색 배경을 만듦
    ImgMasking = np.zeros((img_color.shape[0], img_color.shape[1], 3), np.uint8)

    minX,minY,maxX,maxY, cx, cy = GetMinMaxPosInContour(contours[lastIdx], ImgMasking)

    '''
    cv.circle(ImgMasking, (cx, cy), 10, (0, 255, 255), -1)
    cv.imshow("center point", ImgMasking)
    cv.waitKey(0)
    '''
    # 원본 이미지에서 찾은 컨투어의 좌표를 위에서 만든 검정 배경에 그림
    cv.drawContours(ImgMasking, contours, lastIdx, (255, 255, 255), 2)
    if showFlag:
        cv.imshow("ImgMasking1", ImgMasking)
        cv.waitKey(0)

    # 내부를 흰색으로 채워 줌
    mask = np.zeros((img_color.shape[0] + 2, img_color.shape[1] + 2), np.uint8)
    mask[:] = 0
    cv.floodFill(ImgMasking, mask, (cx, cy), (255, 255, 255))
    #print(cx,cy)

    dst = cv.copyTo(img_color, ImgMasking)

    #print("time :", time.time() - start)
    if showFlag:
        cv.imshow("ImgMasking2", ImgMasking)
        cv.imshow("Last", dst)
        cv.waitKey(0)

    return dst, ImgMasking, contours, lastIdx, cx, cy, minX,minY,maxX,maxY

def IsItOutside(x1, y1, x2, y2):
    '''
    좌표가 알약 외부에 있는지 확인한다.
    :param x1:
    :param y1:
    :param x2:
    :param y2:
    :return: 알약 외부에 있으면 True
    '''
    retValue = False
    if (minX > x1) or (maxX < x1):
        retValue = True
    elif (minX > x2) or (maxX < x2):
        retValue = True
    elif (minY > y1) or (maxY < y1):
        retValue = True
    elif (minY > y2) or (maxY < y2):
        retValue = True

    return  retValue


def CalcPillPositionInfo(FilePath, showFlag = False):
    global foreGroundImg, ImgMasking, contours, lastIdx, cx, cy, minX, minY, maxX, maxY, splitLineInfo, minXOfSplitLine, maxXOfSplitLine, minYOfSplitLine, maxYOfSplitLine

    foreGroundImg, ImgMasking, contours, lastIdx, cx, cy, minX, minY, maxX, maxY = GetForegroundImgInfo(
        FilePath, showFlag)


if __name__ == '__main__':
    import glob
    import os.path as path

    DataParentPath = '/content/'
    fileName = '17159.제스부틴정_crop_001.png'
    # fileName = '15.jpg'
    FilePath = DataParentPath + fileName
    FlagOfExcuteKind = True
    if FlagOfExcuteKind:
        dst, ImgMasking, contours, lastIdx, cx, cy, minX, minY, maxX, maxY = GetForegroundImgInfo(
            FilePath, False)
        cv.waitKey(0)

    else:

        TargetPath = DataParentPath
        FilePathList = glob.glob(path.join(TargetPath, '*.*'))

        for each in FilePathList:
            print(each)

            dst, ImgMasking, contours, lastIdx, cx, cy, minX, minY, maxX, maxY = GetForegroundImgInfo(
                each)

#savePath = DataParentPath + "\\mask\\" + fileName
   #cv.imwrite(savePath, dst)

In [32]:
import enum

class Color(enum.Enum):
    Class0 = 0
    Class1 = 1
    Class2 = 2
    Class3 = 3
    Class4 = 4
    Class5 = 5
    Class6 = 6
    Class7 = 7
    ETC = 99


def GetColor(hsvInfo):
    hue = hsvInfo[0]
    saturation = hsvInfo[1]
    value = hsvInfo[2]

    retValue = Color.ETC

    if saturation < 20 and value > 150:
        retValue = Color.Class0
    elif value < 70:
        retValue = Color.Class1
    elif hue < 10 or hue > 160:
        retValue = Color.Class2
    elif hue <= 40:
        retValue = Color.Class3
    elif hue <= 70:
        retValue = Color.Class4
    elif hue <= 100:
        retValue = Color.Class5
    elif hue <= 130:
        retValue = Color.Class6
    elif hue <= 160:
        retValue = Color.Class7
   
    return retValue

In [33]:
# from Color import Configuration
from sklearn.cluster import KMeans
import cv2 as cv
import PositionManager
import numpy as np

def GetColorHist(img, kClustterValue=2):
    hsvImg = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    reshape = hsvImg.reshape((hsvImg.shape[0] * hsvImg.shape[1]), 3)

    # Find and display most dominant colors
    cluster = KMeans(n_clusters=kClustterValue).fit(reshape)

    # Get the number of different clusters, create histogram, and normalize
    labels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
    (hist, _) = np.histogram(cluster.labels_, bins=labels)
    hist = hist.astype("float")
    hist /= hist.sum()

    # Create frequency rect and iterate through each cluster's color and percentage
    rect = np.zeros((50, 300, 3), dtype=np.uint8)
    colors = sorted([(percent, color) for (percent, color) in zip(hist, cluster.cluster_centers_)])

    return colors, rect


def GetColorInfo(forgroundImg=None, kClustterValue=None, ShowFlag=False):
    global  pillColor
    if forgroundImg is None:
        forgroundImg = PositionManager.foreGroundImg

    if kClustterValue is None:
        kClustterValue = KClustterValue

    pillColor = Color.ETC

    try:
        colors, rect = GetColorHist(forgroundImg, kClustterValue)
    except Exception:
        colors, rect = GetColorHist(forgroundImg, 2)
    start = 0

    lowRatioValueColor = 0.0

    numberOfColors =  len(colors)
    for idx, value in enumerate(colors):
        #print(idx)
        percent, color = value

        # 하위 두개의 색 비율의 합을 구한다.
        if idx <= kClustterValue / 3:
            lowRatioValueColor += percent

        if ShowFlag :
            print(color, "{:0.2f}%".format(percent * 100))

        end = start + (percent * 300)
        cv.rectangle(rect, (int(start), 0), (int(end), 50),
                     color.astype("uint8").tolist(), -1)
        start = end

        # 2번째로 큰 비율의 색을 알약이 색이라 간주
        if idx == (numberOfColors - 2):
            pillColor = GetColor(color)

        # 첫번째로 큰 비율의 색이 검정색이 아니라면 첫번째
        # 큰 비주의 색을 알약의 색이라 간주
        if idx == (numberOfColors - 1):
            # value가 일정값 이상
            if value[1][2] > 5.0:
                pillColor = GetColor(color)

    visualizeData = cv.cvtColor(rect, cv.COLOR_HSV2BGR)
    lowRatioValueColor = lowRatioValueColor * 100.0

    if ShowFlag:
        cv.imshow('color', visualizeData)
        cv.waitKey(0)

    return pillColor, lowRatioValueColor, colors


def AnalizeType(colors, cropedImg):
    minH = 255
    maxH = 0
    minS = 255
    maxS = 0
    minV = 255
    maxV = 0
    for idx, value in enumerate(colors):
        percent, color = value
        currentHue = value[1][0]
        currentSaturation = value[1][1]
        currentValue = value[1][2]
        # hue 값 분석
        if currentHue > maxH:
            maxH = currentHue

        if currentHue < minH:
            minH = currentHue

        if currentSaturation > maxS:
            maxS = currentSaturation

        if currentSaturation < minS:
            minS = currentSaturation

        if currentValue > maxV:
            maxV = currentValue

        if currentValue < minV:
            minV = currentValue

    print(minH, maxH)
    print(minS, maxS)
    print(minV, maxV)
    pillType = 1

    if (maxH - minH) < 40:
        if (maxS - minS < 60):
            if (maxV - minV < 100):
                pillType = 0
        elif (maxV - minV < 60):
            if (maxV - minV < 100):
                pillType = 0
    else:
        hueGap = 180 - (maxH - minH)
        if hueGap < 60:
            if (maxV - minV < 60) and (maxS - minS < 60):
                pillType = 0

    print(pillType)

    return pillType


def TestType(filePath):
    PositionManager.CalcPillPositionInfo(filePath, False)
    img_color = PositionManager.foreGroundImg
    maskImg = PositionManager.ImgMasking
    contours = PositionManager.contours
    lastIdx = PositionManager.lastIdx

    initY = PositionManager.minY + (PositionManager.cy - PositionManager.minY) // 2
    initX = PositionManager.minX + (PositionManager.cx - PositionManager.minX) // 2
    lastY = PositionManager.cy + (PositionManager.maxY - PositionManager.cy) // 2
    lastX = PositionManager.cx + (PositionManager.maxX - PositionManager.cx) // 2
    print(initY, initX, lastY, lastX)
    cropedImgTmp = img_color[initY:lastY, initX:lastX]
    '''
    img_gray = cv.cvtColor(cropedImgTmp, cv.COLOR_BGR2GRAY)
    img_binary_adaptive = cv.adaptiveThreshold(img_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 19,
                                               30)
   # cv.imshow('binary', img_binary_adaptive)
    #cv.waitKey(0)
    rgb = cv.cvtColor(img_binary_adaptive, cv.COLOR_GRAY2RGB)
    '''
    pillColor, sumedLowRatioValue, colors = GetColorInfo(forgroundImg=cropedImgTmp, kClustterValue=3, ShowFlag=False)
    AnalizeType(colors, cropedImgTmp)

    # cv.imshow('crop', cropedImgTmp)
    # cv.waitKey(0)

In [34]:
TestType('/content/17159.제스부틴정_crop_001.png')

111 82 138 128
47.78321678321677 50.43463302752293
11.30069930069915 89.92889908256888
129.38876146788994 156.43006993006995
0


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
